## Machine Learning Model Architecture for Load Characterization and Deformation Prediction

### Overview
The piezoelectric response of our onion peel-based sensor exhibits distinct voltage-time signatures that correlate strongly with both the type of applied load (point vs. uniform) and the magnitude of deformation. We developed a novel two-stage machine learning architecture that leverages these correlations to simultaneously classify load type and predict deformation level from the sensor's electrical output.

### Model Architecture

Our architecture consists of two integrated components that work in sequence:

#### 1. Load Classification Module

The cyclic mechanical loading produces characteristic voltage waveforms that vary distinctly between point and uniform load configurations. To classify these patterns, we treat the voltage-time data as audio signals and employ convolutional neural networks (CNNs) on their time-frequency representations.

**Signal Processing Pipeline:**
- **Data Segmentation**: Continuous voltage-time recordings are segmented into 10-second intervals, each saved as a WAV file
- **Time-Frequency Analysis**: We apply the Short-Time Fourier Transform (STFT) to generate spectrograms:

$$X(\tau, \omega) = \int_{-\infty}^{\infty} x(t) \cdot w(t - \tau) \cdot e^{-j\omega t} \, dt$$

where $x(t)$ is the voltage signal, $w(t-\tau)$ is the window function (Hann window in our implementation), $\tau$ represents the time frame, and $\omega$ is the angular frequency. The resulting complex-valued function $X(\tau, \omega)$ encodes both magnitude and phase information across time and frequency domains.

**Why STFT for Cyclic Loading Analysis:**

The Short-Time Fourier Transform is particularly well-suited for analyzing our piezoelectric sensor data for several key reasons:

<div style="background-color: #f0f8ff; padding: 15px; border-left: 4px solid #4169e1; margin: 15px 0;">

1. **Non-stationary Signal Characteristics**: The voltage response during cyclic compression-decompression is inherently non-stationary—the frequency content changes as the sensor transitions between loading and unloading phases. Unlike a standard Fourier Transform that provides only frequency information, STFT preserves temporal evolution, capturing how spectral components emerge and decay during each loading cycle.

2. **Cycle-Specific Features**: Point and uniform loads produce different stress distributions across the sensor, resulting in distinct harmonic patterns during compression. STFT reveals these time-localized frequency signatures that would be averaged out in a global frequency analysis. For instance, point loads generate sharper transients with higher frequency components at the moment of contact, while uniform loads show more gradual spectral evolution.

3. **Temporal Resolution**: Our 10-second recording windows contain multiple compression cycles. STFT allows us to resolve individual cycles within this window, providing a detailed time-frequency map that the CNN can learn to associate with specific load types. This temporal granularity is crucial for distinguishing between similar voltage amplitudes that arise from different loading conditions.

4. **Robustness to Phase Variations**: Since mechanical loading may not be perfectly synchronized across samples, STFT's sliding window approach provides translation-invariant features that are robust to slight timing variations in the compression cycles.

</div>

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<img src="/home/arisenthil/Desktop/Baroignon/images/stft.gif" alt="STFT Animation" style="width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 1: Short-Time Fourier Transform Visualization.</strong> <em>Real-time generation of time-frequency spectrograms from cyclic voltage signals, showing how frequency content evolves during compression-decompression cycles. The vertical axis represents frequency bins, horizontal axis represents time frames, and color intensity indicates spectral magnitude.</em></p>
</div>

**CNN Architecture:**
The spectrogram images (32×32 pixels after resizing) are processed through a lightweight CNN with the following architecture:
- Two convolutional layers (32 and 64 filters) with ReLU activation
- Max pooling for spatial dimension reduction
- Dropout (rate=0.5) for regularization
- Fully connected layers (128 neurons) leading to binary classification

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<div style="display: inline-block; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); overflow: hidden; max-width: 800px;">
<img src="images/classifierleft.png" alt="CNN Architecture" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0;">
<img src="images/classifierright.png" alt="Spectrogram Examples" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0; border-left: 1px solid #ddd;">
</div>
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 2: Load Classification Pipeline.</strong> <em>Left panel: CNN architecture diagram showing layer-wise operations from spectrogram input to binary classification output. Right panel: Example spectrograms demonstrating distinctive time-frequency patterns for point load (top) and uniform load (bottom) configurations.</em></p>
</div>

Despite the simplicity of this architecture, we achieved [INSERT ACCURACY]% classification accuracy, attributed to the large training dataset (1,440 samples) and the distinctive spectral signatures of the two load types.

#### 2. Deformation Prediction Module: Wide & Deep Regression

Initial experiments using only peak-to-peak voltage as a predictor for deformation showed promising results, revealing an approximately polynomial relationship. However, incorporating the load type as an additional feature in traditional multi-layer perceptron architectures yielded minimal improvement—the categorical nature of the load type was being diluted during training.

To address this limitation, we adapted the Wide & Deep architecture (Cheng et al., 2016), originally developed for recommender systems. This architecture elegantly combines:

- **Wide Component**: A linear model that directly connects the load type (categorical feature) to the output, preserving its discriminative power through memorization
- **Deep Component**: A deep neural network that processes the peak-to-peak voltage through multiple non-linear transformations, enabling generalization across the continuous deformation space

**Architecture Details:**
- **Deep Branch**: Peak-to-peak voltage → Normalization → 4 hidden layers (64, 64, 32, 16 neurons) with ReLU activation
- **Wide Branch**: Load type (binary: 0 for uniform, 1 for point) → Direct connection
- **Fusion**: Concatenation of deep and wide outputs → Linear output layer for deformation prediction

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<img src="images/regression_model.png" alt="Wide & Deep Architecture" style="width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 3: Wide & Deep Regression Architecture for Deformation Prediction.</strong> <em>The model combines a wide component (right) that directly propagates the load type classification to the output, with a deep component (left) that learns non-linear transformations of the peak-to-peak voltage feature. The concatenated features are combined through a final linear layer to predict deformation magnitude.</em></p>
</div>

This architecture leverages the complementary strengths of both features: the wide component maintains the strong correlation between load type and voltage response patterns, while the deep component captures the non-linear relationship between voltage amplitude and deformation magnitude.

### Integration and Inference Pipeline

During inference, the system operates as follows:
1. Input voltage-time data is processed through the CNN classifier, outputting load type probabilities
2. The predicted load type (argmax of probabilities) is converted to binary representation
3. Peak-to-peak voltage is extracted from the raw signal
4. Both features feed into the Wide & Deep regression model for deformation prediction

**Note on Feature Representation:** While the CNN outputs a probability distribution over load types during inference, the regression model uses a discrete binary input (0 or 1) to maintain consistency with the training paradigm where ground truth labels were used.

### Results

The integrated model achieved:
- Load Classification Accuracy: [INSERT VALUE]%
- Deformation Prediction RMSE: [INSERT VALUE] mm
- R² Score: [INSERT VALUE]

These results demonstrate that the synergistic combination of spectral analysis for load classification and Wide & Deep regression for deformation prediction effectively captures the complex electromechanical behavior of our bio-based piezoelectric sensor.

## <span style="color: #ff6f00; border-bottom: 2px solid #ff6f00;">Experimental Data Collection</span>

<table style="width: 100%; border: none;">
<tr>
<td style="width: 100%; vertical-align: top; border: none;">

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">

Piezoelectricity is a function of stress. The mechanical stress (σ) applied to the sensor is related to the deformation through Hooke's law:

<div style="background-color: #fafafa; padding: 15px; border-radius: 5px; text-align: center; margin: 10px 0;">

$$\sigma = \mathbf{E} \cdot \frac{\mathbf{\Delta h}}{\mathbf{h_0}}$$

</div>

<strong style="color: #000;">where:</strong> $\mathbf{E}$ = Young's modulus of the composite material, $\mathbf{\Delta h}$ = change in sensor height, $\mathbf{h_0}$ = original sensor height (2.64 mm)

</div>


<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">
<strong style="color: #ff6f00; font-size: 1.05em;">Experimental Approach:</strong><br>
The piezoelectric response of the onion peel-based sensor is characterized through <strong>cyclic compression testing at three deformation levels</strong> (Δh = 0.5, 0.7, and 1.0 mm) under <strong>two distinct loading configurations</strong> (point and uniform load), with continuous electrical measurements of <strong>Voltage (V) Vs. Time (s)</strong> captured throughout each loading cycle. The image below shows the compression-decompression cycle and the corresponding voltage generation over time.
</div>

<img src="images/compressiongraph.png" alt="Compression Response" style="max-width: 100%; height: auto; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); margin-top: 20px;">
<p style="font-size: 0.9em; color: #666; margin-top: 6px; text-align: center;"><strong>Piezoelectric Response:</strong> <em>(Left) Load vs. displacement during cyclic compression; (Right) Voltage output over time showing peak generation during compression</em></p>

<div style="background-color: #f5f5f5; padding: 12px 15px; border-left: 3px solid #9e9e9e; margin: 10px 0; color: #000;">

<span style="color: #ff6f00; font-weight: bold;">Testing Equipment:</span> Instron tensile testing machine

<span style="color: #ff6f00; font-weight: bold;">Loading Condition:</span> Cyclic compression-release at 100 mm/min

<span style="color: #ff6f00; font-weight: bold;">Sensor Geometry:</span> Original height ($h_0$) = 2.64 mm

</div>

<table style="width: 70%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
<thead>
<tr style="background-color: #fff;">
<th colspan="3" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Deformation Levels</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Δh (mm)</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Compression (% of Original Height)</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Stress Level</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">0.5</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">20%</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Low</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">0.7</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">30%</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Medium</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">1.0</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">40%</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">High</td>
</tr>
</tbody>
</table>

### <span style="color: #ff6f00;">Electrical Signal Acquisition</span>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">

<span style="color: #ff6f00; font-weight: bold;">Measurement Equipment:</span> Keithley 6517B electrometer

<span style="color: #ff6f00; font-weight: bold;">Data Acquisition:</span> Continuous voltage-time recordings during cyclic loading

<span style="color: #ff6f00; font-weight: bold;">Sampling Configuration:</span> Each dataset represents 1 minute of continuous cycling at 100 mm/min

</div>

<table style="width: 80%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
<thead>
<tr style="background-color: #fff;">
<th colspan="5" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Total Recording Duration per Configuration</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Loading Type</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">0.5 mm</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">0.7 mm</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">1.0 mm</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #ff6f00; color: #fff; font-weight: bold;">Total</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Point Load</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">27 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">40 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">53 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #f5f5f5; color: #000; font-weight: bold;">120 min</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Uniform Load</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">27 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">40 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">53 min</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #f5f5f5; color: #000; font-weight: bold;">120 min</td>
</tr>
</tbody>
</table>

### <span style="color: #ff6f00;">Dataset Organization and Structure</span>

<div style="background-color: #fff3e0; padding: 12px 15px; border-left: 3px solid #ff8f00; margin: 10px 0; color: #000;">

To facilitate systematic analysis, continuous voltage-time recordings were segmented into <strong>10-second intervals</strong>, with each interval representing one sample.

</div>

<table style="width: 85%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 6px rgba(0,0,0,0.15);">
<thead>
<tr style="background-color: #fff;">
<th colspan="5" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Final Dataset Distribution</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Loading Type</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">0.5 mm<br><small style="color: #fff;">(20% compression)</small></th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">0.7 mm<br><small style="color: #fff;">(30% compression)</small></th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">1.0 mm<br><small style="color: #fff;">(40% compression)</small></th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #ff6f00; color: #fff; font-weight: bold;">Total Samples</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Point Load</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">160</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">240</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">320</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #f5f5f5; color: #000; font-weight: bold;">720</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Uniform Load</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">160</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">240</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">320</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #f5f5f5; color: #000; font-weight: bold;">720</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Total per Deformation</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">320</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">480</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">640</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; background-color: #e0e0e0; font-size: 1.1em; color: #000; font-weight: bold;">1,440</td>
</tr>
</tbody>
</table>

</td>
</tr>
</table>

## <span style="color: #ff6f00; border-bottom: 2px solid #ff6f00;">Machine Learning Model Architecture for Load Characterization and Deformation Prediction</span>

<table style="width: 100%; border: none;">
<tr>
<td style="width: 100%; vertical-align: top; border: none;">

### <span style="color: #ff6f00;">Overview</span>

<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">
The piezoelectric response of our onion peel-based sensor exhibits <strong>distinct voltage-time signatures</strong> that correlate strongly with both the <strong>type of applied load</strong> (point vs. uniform) and the <strong>magnitude of deformation</strong>. We developed a novel <strong>two-stage machine learning architecture</strong> that leverages these correlations to simultaneously classify load type and predict deformation level from the sensor's electrical output.
</div>

### <span style="color: #ff6f00;">Model Architecture</span>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
Our architecture consists of <strong style="color: #ff6f00;">two integrated components</strong> that work in sequence to achieve comprehensive load characterization and deformation quantification.
</div>

---

### <span style="color: #ff6f00;">1. Load Classification Module</span>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
The cyclic mechanical loading produces <strong>characteristic voltage waveforms</strong> that vary distinctly between point and uniform load configurations. To classify these patterns, we treat the voltage-time data as audio signals and employ <strong>convolutional neural networks (CNNs)</strong> on their time-frequency representations.
</div>

#### <span style="color: #ff6f00;">Signal Processing Pipeline</span>

<div style="background-color: #f5f5f5; padding: 12px 15px; border-left: 3px solid #9e9e9e; margin: 10px 0; color: #000;">

<span style="color: #ff6f00; font-weight: bold;">Data Segmentation:</span> Continuous voltage-time recordings are segmented into 10-second intervals, each saved as a WAV file

<span style="color: #ff6f00; font-weight: bold;">Time-Frequency Analysis:</span> We apply the Short-Time Fourier Transform (STFT) to generate spectrograms

</div>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">

<strong style="color: #ff6f00;">STFT Mathematical Formulation:</strong>

<div style="background-color: #fafafa; padding: 15px; border-radius: 5px; text-align: center; margin: 10px 0;">

$$X(\tau, \omega) = \int_{-\infty}^{\infty} x(t) \cdot w(t - \tau) \cdot e^{-j\omega t} \, dt$$

</div>

<strong style="color: #000;">where:</strong> $x(t)$ = voltage signal, $w(t-\tau)$ = window function (Hann window), $\tau$ = time frame, $\omega$ = angular frequency. The resulting complex-valued function $X(\tau, \omega)$ encodes both magnitude and phase information across time and frequency domains.

</div>

#### <span style="color: #ff6f00;">Why STFT for Cyclic Loading Analysis</span>

<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">

The Short-Time Fourier Transform is particularly well-suited for analyzing our piezoelectric sensor data for several key reasons:

<div style="margin-top: 15px;">

<strong style="color: #ff6f00;">1. Non-stationary Signal Characteristics:</strong><br>
<span style="color: #000;">The voltage response during cyclic compression-decompression is inherently non-stationary—the frequency content changes as the sensor transitions between loading and unloading phases. Unlike a standard Fourier Transform that provides only frequency information, STFT preserves temporal evolution, capturing how spectral components emerge and decay during each loading cycle.</span>

<strong style="color: #ff6f00; margin-top: 10px; display: block;">2. Cycle-Specific Features:</strong><br>
<span style="color: #000;">Point and uniform loads produce different stress distributions across the sensor, resulting in distinct harmonic patterns during compression. STFT reveals these time-localized frequency signatures that would be averaged out in a global frequency analysis. For instance, point loads generate sharper transients with higher frequency components at the moment of contact, while uniform loads show more gradual spectral evolution.</span>

<strong style="color: #ff6f00; margin-top: 10px; display: block;">3. Temporal Resolution:</strong><br>
<span style="color: #000;">Our 10-second recording windows contain multiple compression cycles. STFT allows us to resolve individual cycles within this window, providing a detailed time-frequency map that the CNN can learn to associate with specific load types. This temporal granularity is crucial for distinguishing between similar voltage amplitudes that arise from different loading conditions.</span>

<strong style="color: #ff6f00; margin-top: 10px; display: block;">4. Robustness to Phase Variations:</strong><br>
<span style="color: #000;">Since mechanical loading may not be perfectly synchronized across samples, STFT's sliding window approach provides translation-invariant features that are robust to slight timing variations in the compression cycles.</span>

</div>
</div>

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<img src="/home/arisenthil/Desktop/Baroignon/images/stft.gif" alt="STFT Animation" style="width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 1: Short-Time Fourier Transform Visualization.</strong> <em>Real-time generation of time-frequency spectrograms from cyclic voltage signals, showing how frequency content evolves during compression-decompression cycles. The vertical axis represents frequency bins, horizontal axis represents time frames, and color intensity indicates spectral magnitude.</em></p>
</div>

#### <span style="color: #ff6f00;">CNN Architecture</span>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
The spectrogram images (32×32 pixels after resizing) are processed through a lightweight CNN with the following architecture:
</div>

<table style="width: 80%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
<thead>
<tr style="background-color: #fff;">
<th colspan="2" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">CNN Layer Configuration</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Layer Type</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Specifications</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Convolutional Layers</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">32 and 64 filters with ReLU activation</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Max Pooling</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Spatial dimension reduction</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Dropout</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Rate = 0.5 for regularization</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Fully Connected</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">128 neurons → Binary classification</td>
</tr>
</tbody>
</table>

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<div style="display: inline-block; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); overflow: hidden; max-width: 800px;">
<img src="images/classifierleft.png" alt="CNN Architecture" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0;">
<img src="images/classifierright.png" alt="Spectrogram Examples" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0; border-left: 1px solid #ddd;">
</div>
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 2: Load Classification Pipeline.</strong> <em>Left panel: CNN architecture diagram showing layer-wise operations from spectrogram input to binary classification output. Right panel: Example spectrograms demonstrating distinctive time-frequency patterns for point load (top) and uniform load (bottom) configurations.</em></p>
</div>

<div style="background-color: #fff3e0; padding: 12px 15px; border-left: 3px solid #ff8f00; margin: 10px 0; color: #000;">
Despite the simplicity of this architecture, we achieved <strong>[INSERT ACCURACY]% classification accuracy</strong>, attributed to the large training dataset (1,440 samples) and the distinctive spectral signatures of the two load types.
</div>

---

### <span style="color: #ff6f00;">2. Deformation Prediction Module: Wide & Deep Regression</span>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
Initial experiments using only peak-to-peak voltage as a predictor for deformation showed promising results, revealing an approximately polynomial relationship. However, incorporating the load type as an additional feature in traditional multi-layer perceptron architectures yielded minimal improvement—the categorical nature of the load type was being diluted during training.
</div>

<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">
<strong style="color: #ff6f00; font-size: 1.05em;">Solution: Wide & Deep Architecture</strong><br>
To address this limitation, we adapted the Wide & Deep architecture (Cheng et al., 2016), originally developed for recommender systems. This architecture elegantly combines <strong>memorization</strong> (wide component) with <strong>generalization</strong> (deep component).
</div>

#### <span style="color: #ff6f00;">Architecture Components</span>

<table style="width: 85%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 6px rgba(0,0,0,0.15);">
<thead>
<tr style="background-color: #fff;">
<th colspan="3" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Wide & Deep Model Structure</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Component</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Input Features</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Function</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Wide Component</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Load type (binary: 0 or 1)</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Direct connection → Preserves discriminative power through memorization</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; background-color: #f9f9f9; text-align: center; color: #000; font-weight: bold;">Deep Component</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Peak-to-peak voltage</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">Multiple non-linear transformations → Enables generalization</td>
</tr>
</tbody>
</table>

<div style="background-color: #f5f5f5; padding: 12px 15px; border-left: 3px solid #9e9e9e; margin: 10px 0; color: #000;">

<span style="color: #ff6f00; font-weight: bold;">Deep Branch Architecture:</span> Peak-to-peak voltage → Normalization → 4 hidden layers (64, 64, 32, 16 neurons) with ReLU activation

<span style="color: #ff6f00; font-weight: bold;">Wide Branch Architecture:</span> Load type (binary: 0 for uniform, 1 for point) → Direct connection

<span style="color: #ff6f00; font-weight: bold;">Fusion Layer:</span> Concatenation of deep and wide outputs → Linear output layer for deformation prediction

</div>

<div style="text-align: center; margin: 25px auto; max-width: 90%;">
<img src="images/regression_model.png" alt="Wide & Deep Architecture" style="width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Figure 3: Wide & Deep Regression Architecture for Deformation Prediction.</strong> <em>The model combines a wide component (right) that directly propagates the load type classification to the output, with a deep component (left) that learns non-linear transformations of the peak-to-peak voltage feature. The concatenated features are combined through a final linear layer to predict deformation magnitude.</em></p>
</div>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
This architecture leverages the complementary strengths of both features: the <strong style="color: #ff6f00;">wide component</strong> maintains the strong correlation between load type and voltage response patterns, while the <strong style="color: #ff6f00;">deep component</strong> captures the non-linear relationship between voltage amplitude and deformation magnitude.
</div>

---

### <span style="color: #ff6f00;">Integration and Inference Pipeline</span>

<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">
<strong style="color: #ff6f00; font-size: 1.05em;">Inference Process:</strong><br>
During inference, the system operates through a sequential pipeline that transforms raw sensor data into quantitative load characterization and deformation prediction.
</div>

<table style="width: 90%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 6px rgba(0,0,0,0.15);">
<thead>
<tr style="background-color: #fff;">
<th colspan="2" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Sequential Inference Steps</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Step</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Process</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">1</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: left; color: #000;">Input voltage-time data is processed through the CNN classifier, outputting load type probabilities</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">2</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: left; color: #000;">The predicted load type (argmax of probabilities) is converted to binary representation</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">3</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: left; color: #000;">Peak-to-peak voltage is extracted from the raw signal</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">4</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: left; color: #000;">Both features feed into the Wide & Deep regression model for deformation prediction</td>
</tr>
</tbody>
</table>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
<strong style="color: #ff6f00;">Note on Feature Representation:</strong> While the CNN outputs a probability distribution over load types during inference, the regression model uses a discrete binary input (0 or 1) to maintain consistency with the training paradigm where ground truth labels were used.
</div>

---

### <span style="color: #ff6f00;">Results and Performance Metrics</span>

<div style="background-color: #fff3e0; padding: 15px 20px; border-left: 5px solid #ff6f00; border-radius: 5px; margin: 15px 0; box-shadow: 0 2px 8px rgba(255,111,0,0.2); color: #000;">
The integrated model demonstrates <strong>robust performance</strong> across both classification and regression tasks, validating the effectiveness of the two-stage architecture.
</div>

<table style="width: 75%; margin: 15px auto; border-collapse: collapse; box-shadow: 0 2px 6px rgba(0,0,0,0.15);">
<thead>
<tr style="background-color: #fff;">
<th colspan="2" style="padding: 15px; border: 1px solid #ddd; text-align: center; color: #ff6f00; font-weight: bold; font-size: 1.1em; background-color: #fff;">Model Performance Summary</th>
</tr>
<tr style="background-color: #ff8f00;">
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Metric</th>
<th style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #fff; font-weight: bold;">Value</th>
</tr>
</thead>
<tbody>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Load Classification Accuracy</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">[INSERT VALUE]%</td>
</tr>
<tr style="background-color: #f5f5f5;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">Deformation Prediction RMSE</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">[INSERT VALUE] mm</td>
</tr>
<tr style="background-color: #fff;">
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000; font-weight: bold;">R² Score</td>
<td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: #000;">[INSERT VALUE]</td>
</tr>
</tbody>
</table>

<div style="background-color: #fff; padding: 12px 15px; border: 1px solid #bdbdbd; border-radius: 5px; margin: 10px 0; color: #000;">
These results demonstrate that the <strong style="color: #ff6f00;">synergistic combination</strong> of spectral analysis for load classification and Wide & Deep regression for deformation prediction effectively captures the complex electromechanical behavior of our bio-based piezoelectric sensor.
</div>

</td>
</tr>
</table>

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Machine Learning Model Architecture for Load Characterization and Deformation Prediction</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
The piezoelectric response of our onion peel-based sensor exhibits distinct voltage-time signatures that correlate strongly with both the type of applied load (point vs. uniform) and the magnitude of deformation. We developed a novel <strong>two-stage machine learning architecture</strong> that leverages these correlations to simultaneously classify load type and predict deformation level from the sensor's electrical output.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Two-Stage Model Architecture</span>

<table style="width: 100%; border: none;">
<tr>
<td style="width: 60%; vertical-align: top; padding-right: 20px; border: none;">

### <span style="color: #3498db;">1. Load Classification Module</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
The cyclic mechanical loading produces characteristic voltage waveforms that vary distinctly between point and uniform load configurations. To classify these patterns, we treat the voltage-time data as audio signals and employ <strong>convolutional neural networks (CNNs)</strong> on their time-frequency representations.
</div>

### <span style="color: #3498db;">Signal Processing Pipeline</span>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>Data Segmentation:</strong> Continuous voltage-time recordings are segmented into 10-second intervals, each saved as a WAV file<br><br>
<strong>Time-Frequency Analysis:</strong> We apply the Short-Time Fourier Transform (STFT) to generate spectrograms
</div>

### <span style="color: #3498db;">Mathematical Expression</span>

<div style="background-color: #ecf0f1; padding: 15px; border-radius: 5px; text-align: center; margin: 8px 0;">

$$X(\tau, \omega) = \int_{-\infty}^{\infty} x(t) \cdot w(t - \tau) \cdot e^{-j\omega t} \, dt$$

</div>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>where:</strong> $x(t)$ = voltage signal, $w(t-\tau)$ = window function (Hann window), $\tau$ = time frame, $\omega$ = angular frequency
</div>

</td>
<td style="width: 40%; vertical-align: top; border: none;">

<img src="/home/arisenthil/Desktop/Baroignon/images/stft.gif" alt="STFT Animation" style="max-width: 100%; height: auto; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 6px; text-align: center;"><strong>STFT Visualization:</strong> <em>Real-time generation of time-frequency spectrograms from cyclic voltage signals</em></p>

</td>
</tr>
</table>

### <span style="color: #3498db;">Why STFT for Cyclic Loading Analysis?</span>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Non-stationary Signal Characteristics:</strong> The voltage response during cyclic compression-decompression is inherently non-stationary—the frequency content changes as the sensor transitions between loading and unloading phases. STFT preserves temporal evolution, capturing how spectral components emerge and decay during each loading cycle.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Cycle-Specific Features:</strong> Point and uniform loads produce different stress distributions across the sensor, resulting in distinct harmonic patterns during compression. STFT reveals these time-localized frequency signatures. Point loads generate sharper transients with higher frequency components at the moment of contact, while uniform loads show more gradual spectral evolution.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Temporal Resolution:</strong> Our 10-second recording windows contain multiple compression cycles. STFT allows us to resolve individual cycles within this window, providing a detailed time-frequency map that the CNN can learn to associate with specific load types.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Robustness to Phase Variations:</strong> Since mechanical loading may not be perfectly synchronized across samples, STFT's sliding window approach provides translation-invariant features that are robust to slight timing variations in the compression cycles.
</div>

### <span style="color: #3498db;">CNN Architecture</span>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
The spectrogram images (32×32 pixels after resizing) are processed through a lightweight CNN with the following architecture:<br><br>
• Two convolutional layers (32 and 64 filters) with ReLU activation<br>
• Max pooling for spatial dimension reduction<br>
• Dropout (rate=0.5) for regularization<br>
• Fully connected layers (128 neurons) leading to binary classification
</div>

<div style="text-align: center; margin: 25px auto;">
<div style="display: inline-block; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); overflow: hidden; max-width: 800px;">
<img src="images/classifierleft.png" alt="CNN Architecture" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0;">
<img src="images/classifierright.png" alt="Spectrogram Examples" style="display: inline-block; vertical-align: top; width: 50%; margin: 0; padding: 0; border: 0; border-left: 1px solid #ddd;">
</div>
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Load Classification Pipeline:</strong> <em>Left: CNN architecture diagram; Right: Example spectrograms for point load (top) and uniform load (bottom)</em></p>
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">2. Deformation Prediction Module: Wide & Deep Regression</span>

<table style="width: 100%; border: none;">
<tr>
<td style="width: 60%; vertical-align: top; padding-right: 20px; border: none;">

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
Initial experiments using only peak-to-peak voltage as a predictor for deformation showed promising results. However, incorporating the load type as an additional feature in traditional multi-layer perceptron architectures yielded minimal improvement—the categorical nature of the load type was being diluted during training.
</div>

### <span style="color: #3498db;">Wide & Deep Architecture</span>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
We adapted the Wide & Deep architecture (Cheng et al., 2016), originally developed for recommender systems. This architecture elegantly combines memorization and generalization.
</div>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>Wide Component:</strong> A linear model that directly connects the load type (categorical feature) to the output, preserving its discriminative power through memorization<br><br>
<strong>Deep Component:</strong> A deep neural network that processes the peak-to-peak voltage through multiple non-linear transformations, enabling generalization across the continuous deformation space
</div>

### <span style="color: #3498db;">Architecture Details</span>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>Deep Branch:</strong> Peak-to-peak voltage → Normalization → 4 hidden layers (64, 64, 32, 16 neurons) with ReLU activation<br><br>
<strong>Wide Branch:</strong> Load type (binary: 0 for uniform, 1 for point) → Direct connection<br><br>
<strong>Fusion:</strong> Concatenation of deep and wide outputs → Linear output layer for deformation prediction
</div>

</td>
<td style="width: 40%; vertical-align: top; border: none;">

<img src="images/regression_model.png" alt="Wide & Deep Architecture" style="max-width: 100%; height: auto; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 6px; text-align: center;"><strong>Wide & Deep Architecture:</strong> <em>Combining wide component (right) with deep component (left) for deformation prediction</em></p>

</td>
</tr>
</table>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
This architecture leverages the complementary strengths of both features: the wide component maintains the strong correlation between load type and voltage response patterns, while the deep component captures the non-linear relationship between voltage amplitude and deformation magnitude.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Integration and Inference Pipeline</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
During inference, the system operates as follows:<br><br>
<strong>1.</strong> Input voltage-time data is processed through the CNN classifier, outputting load type probabilities<br>
<strong>2.</strong> The predicted load type (argmax of probabilities) is converted to binary representation<br>
<strong>3.</strong> Peak-to-peak voltage is extracted from the raw signal<br>
<strong>4.</strong> Both features feed into the Wide & Deep regression model for deformation prediction
</div>

<div style="background-color: #fff3e0; padding: 12px 15px; border-left: 3px solid #ff9800; margin: 10px 0;">
<strong>Note on Feature Representation:</strong> While the CNN outputs a probability distribution over load types during inference, the regression model uses a discrete binary input (0 or 1) to maintain consistency with the training paradigm where ground truth labels were used.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Results</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #4caf50; margin: 10px 0;">
The integrated model achieved:<br><br>
<strong>Load Classification Accuracy:</strong> [INSERT VALUE]%<br>
<strong>Deformation Prediction RMSE:</strong> [INSERT VALUE] mm<br>
<strong>R² Score:</strong> [INSERT VALUE]
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
These results demonstrate that the synergistic combination of spectral analysis for load classification and Wide & Deep regression for deformation prediction effectively captures the complex electromechanical behavior of our bio-based piezoelectric sensor.
</div>

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Machine Learning Model Architecture for Load Characterization and Deformation Prediction</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
The piezoelectric response of our onion peel-based sensor exhibits distinct voltage-time signatures that correlate strongly with both the type of applied load (point vs. uniform) and the magnitude of deformation. We developed a novel <strong>two-stage machine learning architecture</strong> that leverages these correlations to simultaneously classify load type and predict deformation level from the sensor's electrical output.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Two-Stage Model Architecture</span>

<table style="width: 100%; border: none;">
<tr>
<td style="width: 60%; vertical-align: top; padding-right: 20px; border: none;">

### <span style="color: #3498db;">1. Load Classification Module</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
The cyclic mechanical loading produces characteristic voltage waveforms that vary distinctly between point and uniform load configurations. To classify these patterns, we treat the voltage-time data as audio signals and employ <strong>convolutional neural networks (CNNs)</strong> on their time-frequency representations.
</div>

### <span style="color: #3498db;">Signal Processing Pipeline</span>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>Data Segmentation:</strong> Continuous voltage-time recordings are segmented into 10-second intervals, each saved as a WAV file<br><br>
<strong>Time-Frequency Analysis:</strong> We apply the Short-Time Fourier Transform (STFT) to generate spectrograms
</div>

### <span style="color: #3498db;">Mathematical Expression</span>

<div style="background-color: #ecf0f1; padding: 15px; border-radius: 5px; text-align: center; margin: 8px 0;">

$$X(\tau, \omega) = \int_{-\infty}^{\infty} x(t) \cdot w(t - \tau) \cdot e^{-j\omega t} \, dt$$

</div>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>where:</strong> $x(t)$ = voltage signal, $w(t-\tau)$ = window function (Hann window), $\tau$ = time frame, $\omega$ = angular frequency
</div>

</td>
<td style="width: 40%; vertical-align: top; border: none;">

<img src="/home/arisenthil/Desktop/Baroignon/images/stft.gif" alt="STFT Animation" style="max-width: 100%; height: auto; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">

<p style="font-size: 0.9em; color: #666; margin-top: 6px; text-align: center;"><strong>STFT Visualization:</strong> <em>Real-time generation of time-frequency spectrograms from cyclic voltage signals</em></p>

</td>
</tr>
</table>

### <span style="color: #3498db;">Why STFT for Cyclic Loading Analysis?</span>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Non-stationary Signal Characteristics:</strong> The voltage response during cyclic compression-decompression is inherently non-stationary—the frequency content changes as the sensor transitions between loading and unloading phases. STFT preserves temporal evolution, capturing how spectral components emerge and decay during each loading cycle.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Cycle-Specific Features:</strong> Point and uniform loads produce different stress distributions across the sensor, resulting in distinct harmonic patterns during compression. STFT reveals these time-localized frequency signatures. Point loads generate sharper transients with higher frequency components at the moment of contact, while uniform loads show more gradual spectral evolution.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Temporal Resolution:</strong> Our 10-second recording windows contain multiple compression cycles. STFT allows us to resolve individual cycles within this window, providing a detailed time-frequency map that the CNN can learn to associate with specific load types.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
<strong>Robustness to Phase Variations:</strong> Since mechanical loading may not be perfectly synchronized across samples, STFT's sliding window approach provides translation-invariant features that are robust to slight timing variations in the compression cycles.
</div>

### <span style="color: #3498db;">CNN Architecture</span>


<div style="text-align: center; margin: 25px auto;">
<img src="/home/ari/Documents/Onion_Sensor/images/Onion Ink/cnn.png" alt="Wide & Deep Architecture" style="max-width: 100%; width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Load Classification Architecture:</strong> <em>CNN that classifies point or uniform load </em></p>
</div>


### <span style="color: #3498db;">2. Deformation Prediction Module: Wide & Deep Regression</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
Initial experiments using only peak-to-peak voltage as a predictor for deformation showed promising results. However, incorporating the load type as an additional feature in traditional multi-layer perceptron architectures yielded minimal improvement—the categorical nature of the load type was being diluted during training.
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
We adapted the <strong>Wide & Deep architecture</strong> (Cheng et al., 2016), originally developed for recommender systems. This architecture elegantly combines memorization and generalization.
</div>

<div style="background-color: #fff; padding: 10px 15px; border: 1px solid #ddd; border-radius: 5px; margin: 8px 0;">
<strong>Wide Component:</strong> A linear model that directly connects the load type (categorical feature) to the output, preserving its discriminative power through memorization<br><br>
<strong>Deep Component:</strong> A deep neural network that processes the peak-to-peak voltage through multiple non-linear transformations, enabling generalization across the continuous deformation space
</div>


<div style="text-align: center; margin: 25px auto;">
<img src="/home/ari/Documents/Onion_Sensor/images/Onion Ink/wide.png" alt="Wide & Deep Architecture" style="max-width: 100%; width: 100%; max-width: 800px; border: 2px solid #ddd; border-radius: 5px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
<p style="font-size: 0.9em; color: #666; margin-top: 8px;"><strong>Wide & Deep Regression Architecture:</strong> <em>Combining wide component (right) with deep component (left) for deformation prediction</em></p>
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
This architecture leverages the complementary strengths of both features: the wide component maintains the strong correlation between load type and voltage response patterns, while the deep component captures the non-linear relationship between voltage amplitude and deformation magnitude.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Integration and Inference Pipeline</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #3498db; margin: 10px 0;">
During inference, the system operates as follows:<br><br>
<strong>1.</strong> Input voltage-time data is processed through the CNN classifier, outputting load type probabilities<br>
<strong>2.</strong> The predicted load type (argmax of probabilities) is converted to binary representation<br>
<strong>3.</strong> Peak-to-peak voltage is extracted from the raw signal<br>
<strong>4.</strong> Both features feed into the Wide & Deep regression model for deformation prediction
</div>

<div style="background-color: #fff3e0; padding: 12px 15px; border-left: 3px solid #ff9800; margin: 10px 0;">
<strong>Note on Feature Representation:</strong> While the CNN outputs a probability distribution over load types during inference, the regression model uses a discrete binary input (0 or 1) to maintain consistency with the training paradigm where ground truth labels were used.
</div>

---

## <span style="color: rgb(2, 14, 26); border-bottom: 2px solid #3498db;">Results</span>

<div style="background-color: #f8f9fa; padding: 12px 15px; border-left: 3px solid #4caf50; margin: 10px 0;">
The integrated model achieved:<br><br>
<strong>Load Classification Accuracy:</strong> [INSERT VALUE]%<br>
<strong>Deformation Prediction RMSE:</strong> [INSERT VALUE] mm<br>
<strong>R² Score:</strong> [INSERT VALUE]
</div>

<div style="background-color: #e8f4f8; padding: 12px 15px; border-left: 3px solid #2980b9; margin: 10px 0;">
These results demonstrate that the synergistic combination of spectral analysis for load classification and Wide & Deep regression for deformation prediction effectively captures the complex electromechanical behavior of our bio-based piezoelectric sensor.
</div>